# Fitting SFG Curves with Multiple Resonant Peaks
Using the iminuit package, select a window of SFG spectra and perform a non-linear fit of nonresonant and a variable number ($N^{res}$) of resonant peaks accoring to the equation
$$ \mathrm{SFG}(\omega) = \left| A^{nonres}e^{i\phi} + \sum_{j=0}^{N^{res}}\frac{A^{res}_j}{\omega - \omega^{res}_j+i\Gamma_j}\right|^2 $$
where the parameters that we want to determine are the nonresonant amplitue ($A^{nonres}$) and phase ($\phi$), as well as the amplitude, position, and width of each resonant peak ($A^{res}_j$, $\omega^{res}_j$, and $\Gamma_j$, respectively).

Developers : Oliviero Andreussi, Lindsey Jenkins, Tiara Sivells, Pranav Viswanathan, Jenee Cyran


## Mount the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import External Modules

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
!pip install iminuit
from iminuit import Minuit

## Define Basic Functions for the Fitting

In [ ]:
# Basic functions for the fitting of nonresonant and resonant peaks
def chi_non_resonant(nr: float, phase: float) -> np.complex128 :
  """
  Given the non-resonant parameters return a single complex-valued number
  for the non-resonant process
  """
  ChiNR = nr * np.exp(1j * phase)
  return ChiNR

def chi_resonant(wavenumbers: np.ndarray[np.float64], amplitude: float, pos: float, width: float) -> np.ndarray[np.complex128]:
  """
  Given a range of wavenumbers and the parameters of a resonant peak return
  the complex values of the peak for each wavenumber
  """
  A = amplitude
  delta = wavenumbers - pos
  gamma = width / 2
  ChiR_i = -(A * gamma / (delta**2 + gamma**2))
  ChiR_r = A * delta / (delta**2 + gamma**2)
  ChiR = ChiR_r + (1j * ChiR_i)
  return ChiR

## Load Data

In [ ]:
# @title Set path and select file { display-mode: "form" }
# Data should be cleaned using MATLAB data cleaner first
path = '/content/drive/MyDrive/Colab Notebooks/' # @param {type:"string"}
x = pd.read_csv(path+'xaxis.csv',names=['Wavenumbers'],skiprows=1) #skips header
y = pd.read_csv(path+'yaxis.csv',names=['SFG'],skiprows=1) #change name to what sample it is

# Uncomment to make sure that the size of the arrays are the same size
# x.shape
# y.shape
# Makes the arrays into a data structure
data = pd.concat([x,y],axis=1)

## Plot of data

In [ ]:
data.plot('Wavenumbers', 'SFG')

In [ ]:
# @title Resize the window of the spectrum { display-mode: "form" }
WMin = 2700 # @param {type:"number"}
WMax  = 3500 # @param {type:"number"}
filtered_data = data.query(f'Wavenumbers > {WMin} and Wavenumbers < {WMax}')
# Plots the data with the new range
filtered_data.plot('Wavenumbers','SFG')
wavenumbers = filtered_data['Wavenumbers'].values
sfg = filtered_data['SFG'].values

## Fit a Single Resonant Peak

In [ ]:
# Parameters for each peak
# Change to parameters in water
nr = { "amplitude": 0.10399,
       "phase": np.pi }
r0 = { "amplitude" : 2,
       "pos" : 3270.,
       "width" : 20 }
r1 = { "amplitude": 16.1201,
       "pos": 3325,
       "width": 200 }

In [ ]:
chi_non_resonant(nr['amplitude'],nr['phase'])

In [ ]:
chi_resonant(filtered_data['Wavenumbers'],r0['amplitude'],r0['pos'],r0['width'])

In [ ]:
# functions to fit and cost functions, with explicit parameters
# NOTE: these functions use wavenumbers and sfg defined in the global
# scope of the notebook

def calcamplitude(nr, phase, amplitude, pos, width):
  ChiNR = chi_non_resonant(nr, phase)
  ChiR = chi_resonant(wavenumbers, amplitude, pos, width)
  Chi = ChiNR + ChiR
  return np.square(Chi.real) + np.square(Chi.imag)

def costfunction(nr, phase, amplitude, pos, width):
  return np.sum((sfg - calcamplitude(nr, phase, amplitude, pos, width))**2)

In [ ]:
calcamplitude(nr['amplitude'],nr['phase'],r0['amplitude'],r0['pos'],r0['width'])

Trying to see if the parameters we chose are reasonable for the data

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcamplitude(nr['amplitude'],nr['phase'],r0['amplitude'],r0['pos'],r0['width']))
plt.plot(filtered_data['Wavenumbers'],filtered_data['SFG'])

In [ ]:
# @title Adjust the initial parameters to check the convergence of the fit { display-mode: "form" }
nonres_amplitude = 0.1 # @param {type:"number"}
nonres_phase  = 3.14# @param {type:"number"}
res_amplitude = 0.005 # @param {type:"number"}
res_pos = 3200 # @param {type:"number"}
res_width = 30 # @param {type:"number"}

fitting_args = {'nr': nonres_amplitude, 'phase': nonres_phase, 'amplitude': res_amplitude, 'pos': res_pos, 'width': res_width}
fit = Minuit(costfunction, **fitting_args)
# Ranges should only be positive
fit.limits["nr"] = (0, None)
fit.limits["phase"] = (0, 2*np.pi)
fit.limits["amplitude"] = (0, None)
fit.limits["pos"] = (WMin, WMax)
fit.limits["width"] = (0, None)

# perform the fit
fit.migrad()

# plot result of fit with optimized parameters vs. experiment
plt.plot(filtered_data['Wavenumbers'],calcamplitude(fit.params[0].value,fit.params[1].value,fit.params[2].value,fit.params[3].value,fit.params[4].value))
plt.plot(filtered_data['Wavenumbers'],filtered_data['SFG'])

This step performs the actual optmization of the parameters (the results look cool, not sure what all these numbers mean...)

In [ ]:
fit.migrad()

Initial parameters

In [ ]:
fit.init_params

Optimized parameters

In [ ]:
fit.params

We can access the final value and associated error with the .value and .error attributes

In [ ]:
print(fit.params[0].value,fit.params[0].error)

Now we can reuse the calcamplitude function with the optimal parameters to compare to the experimental data

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcamplitude(fit.params[0].value,fit.params[1].value,fit.params[2].value,fit.params[3].value,fit.params[4].value))
plt.plot(filtered_data['Wavenumbers'],filtered_data['SFG'])

## Still only one resonant peak, but with a more flexible implementation

This creates a new dictionary with all the parameters from the individual components we want to fit

In [ ]:
maxnres = 5 # global parameter with the maximum number of resonant peaks that we will ever need

In [ ]:
fitting_dictionaries = { 'nr' : nr, 'r0' : r0 }
parameters = {}
for label, dictionary in fitting_dictionaries.items() :
  new = { label+'_'+k:v for k, v in dictionary.items()}
  parameters = {**parameters,**new}
print(parameters)

In [ ]:
parameters.keys()

In [ ]:
wavenumbers = filtered_data['Wavenumbers'].values
sfg = filtered_data['SFG'].values
# This implementation has a flexible fitting function, but the cost function still needs explicit parameters
def calcamplitude(**kwds):
  Chi = np.zeros(sfg.shape,dtype=np.complex128)
  if 'nr_amplitude' in kwds.keys():
    ChiNR = chi_non_resonant(kwds['nr_amplitude'], kwds['nr_phase'])
    Chi = Chi + ChiNR
  for i in range(maxnres):
    if 'r'+str(i)+'_amplitude' in kwds.keys():
       ChiR = chi_resonant(wavenumbers, kwds['r'+str(i)+'_amplitude'], kwds['r'+str(i)+'_pos'], kwds['r'+str(i)+'_width'])
       Chi = Chi + ChiR
  return np.square(Chi.real) + np.square(Chi.imag)

def cost0(nr_amplitude, nr_phase, r0_amplitude, r0_pos, r0_width):
  return np.sum((sfg - calcamplitude(nr_amplitude=nr_amplitude, nr_phase=nr_phase, \
                                     r0_amplitude=r0_amplitude, r0_pos=r0_pos, r0_width=r0_width))**2)


In [ ]:
chi_non_resonant(0.10399, np.pi)

In [ ]:
chi_resonant(wavenumbers, parameters['r0_amplitude'], parameters['r0_pos'], parameters['r0_width'])

In [ ]:
calcamplitude(**parameters)

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcamplitude(**parameters))
plt.plot(filtered_data['Wavenumbers'],filtered_data['SFG'])

In [ ]:
fit = Minuit(cost0, **parameters)
# Ranges should only be positive
if 'nr_amplitude' in parameters:
  fit.limits["nr_amplitude"] = (0, None)
  fit.limits["nr_phase"] = (0, 2*np.pi)
for i in range(maxnres):
  if 'r'+str(i)+'_amplitude' in parameters:
    fit.limits['r'+str(i)+'_amplitude'] = (0, None)
    fit.limits['r'+str(i)+'_pos'] = (WMin, WMax)
    fit.limits['r'+str(i)+'_width'] = (0, None)


In [ ]:
# perform the fit
fit.migrad()

## More than one resonant peak

In [ ]:
# @title Resize the window of the spectrum { display-mode: "form" }
WMin = 2740 # @param {type:"number"}
WMax  = 3400 # @param {type:"number"}
filtered_data = data.query(f'Wavenumbers > {WMin} and Wavenumbers < {WMax}').copy()
filtered_data.plot('Wavenumbers','SFG')
wavenumbers = filtered_data['Wavenumbers'].values
sfg = filtered_data['SFG'].values

In [ ]:
# Parameters for each peak
nr = { "amplitude": 0.10399,
       "phase": np.pi }
r0 = { "amplitude" : 3,
       "pos" : 2870.,
       "width" : 20 }
r1 = { "amplitude": 7,
       "pos": 2800,
       "width": 50 }

In [ ]:
fitting_dictionaries = { 'nr' : nr, 'r0' : r0, 'r1' : r1 }
parameters = {}
for label, dictionary in fitting_dictionaries.items() :
  new = { label+'_'+k:v for k, v in dictionary.items()}
  parameters = {**parameters,**new}
print(parameters)

In [ ]:
def calcamplitude(**kwds):
  Chi = np.zeros(sfg.shape,dtype=np.complex128)
  if 'nr_amplitude' in kwds.keys():
    ChiNR = chi_non_resonant(kwds['nr_amplitude'], kwds['nr_phase'])
    Chi = Chi + ChiNR
  for i in range(maxnres):
    if 'r'+str(i)+'_amplitude' in kwds.keys():
       ChiR = chi_resonant(wavenumbers, kwds['r'+str(i)+'_amplitude'], kwds['r'+str(i)+'_pos'], kwds['r'+str(i)+'_width'])
       Chi = Chi + ChiR
  return np.square(Chi.real) + np.square(Chi.imag)

def calcimaginary(**kwds):
  Chi = np.zeros(sfg.shape,dtype=np.complex128)
  if 'nr_amplitude' in kwds.keys():
    ChiNR = chi_non_resonant(kwds['nr_amplitude'], kwds['nr_phase'])
    Chi = Chi + ChiNR
  for i in range(maxnres):
    if 'r'+str(i)+'_amplitude' in kwds.keys():
       ChiR = chi_resonant(wavenumbers, kwds['r'+str(i)+'_amplitude'], kwds['r'+str(i)+'_pos'], kwds['r'+str(i)+'_width'])
       Chi = Chi + ChiR
  return Chi.imag

def cost0(nr_amplitude, nr_phase, r0_amplitude, r0_pos, r0_width):
  return np.sum((sfg - calcamplitude(nr_amplitude=nr_amplitude, nr_phase=nr_phase, \
                                     r0_amplitude=r0_amplitude, r0_pos=r0_pos, r0_width=r0_width))**2)

def cost1(nr_amplitude, nr_phase, r0_amplitude, r0_pos, r0_width, r1_amplitude, r1_pos, r1_width ):
  return np.sum((sfg - calcamplitude(nr_amplitude=nr_amplitude, nr_phase=nr_phase, \
                                     r0_amplitude=r0_amplitude, r0_pos=r0_pos, r0_width=r0_width,\
                                     r1_amplitude=r1_amplitude, r1_pos=r1_pos, r1_width=r1_width\
                                     ))**2)

In [ ]:
fit = Minuit(cost1, **parameters)
# Ranges should only be positive
if 'nr_amplitude' in parameters:
  fit.limits["nr_amplitude"] = (0, None)
  fit.limits["nr_phase"] = (0, 2*np.pi)
for i in range(maxnres):
  if 'r'+str(i)+'_amplitude' in parameters:
    fit.limits['r'+str(i)+'_amplitude'] = (0, None)
    fit.limits['r'+str(i)+'_pos'] = (WMin, WMax)
    fit.limits['r'+str(i)+'_width'] = (0, None)

In [ ]:
fit.fixed["nr_phase"] = False
fit.fixed["r0_pos"] = False
fit.fixed["r1_pos"] = False

In [ ]:
# perform the fit
fit.migrad()

Create a new dictionary with all the optimized parameters (a better version than before)

In [ ]:
optimized_parameters = dict(zip(parameters.keys(),[p.value for p in fit.params]))
optimized_parameters

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcamplitude(**optimized_parameters))
plt.plot(filtered_data['Wavenumbers'],filtered_data['SFG'])

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcimaginary(**optimized_parameters))

## New attempt at a general cost function

Minuit documentation `help(Minuit)` explains that we can use costfunctions that depends on a variable number of arguments, provided they are positional arguments, I have a feeling that it does not work for keyword arguments. Thus, we can test using positional arguments `*args`, but for the time being we will assume that the arguments are ordered according to what we have done above (the first two are nonresonant parameters, then we have three parameters for each resonant function to add).

In [ ]:
def calcamplitude(*args):
  Chi = np.zeros(sfg.shape,dtype=np.complex128)
  Chi = Chi + chi_non_resonant(args[0], args[1])
  nres = (len(args)-2)//3
  for i in range(nres):
    iarg = 3*i+2
    ChiR = chi_resonant(wavenumbers, args[iarg], args[iarg+1], args[iarg+2])
    Chi = Chi + ChiR
  return np.square(Chi.real) + np.square(Chi.imag)

def calcimaginary(*args):
  Chi = np.zeros(sfg.shape,dtype=np.complex128)
  Chi = Chi + chi_non_resonant(args[0], args[1])
  nres = (len(args)-2)//3
  for i in range(nres):
    iarg = 3*i+2
    ChiR = chi_resonant(wavenumbers, args[iarg], args[iarg+1], args[iarg+2])
    Chi = Chi + ChiR
  return Chi.imag

def costfunction(*args):
  return np.sum((sfg - calcamplitude(*args))**2)

costfunction.errordef = Minuit.LEAST_SQUARES

 NOTE: from Minuit documentation it explains that if the costfunction is a least-squares type, it should have an attribute fcn.errordef set to 1. I believe this is the default value, so it should not matter if we set it or not, but it does no harm to set it. Also, Minuit.LEAST_SQUARES is just a constant equal to 1, to be used to this purpose.

```
 |  errordef
 |      Access FCN increment above minimum that corresponds to one standard deviation.
 |      
 |      Default value is 1.0. `errordef` should be 1.0 for a least-squares cost function
 |      and 0.5 for a negative log-likelihood function. See section 1.5.1 on page 6 of
 |      the :download:`MINUIT2 User's Guide <mnusersguide.pdf>`. This parameter is also
 |      called *UP* in MINUIT documents.
 |      
 |      If FCN has an attribute ``errordef``, its value is used automatically and you
 |      should not set errordef by hand. Doing so will raise a
 |      ErrordefAlreadySetWarning.
 |      
 |      For the builtin cost functions in :mod:`iminuit.cost`, you don't need to set
 |      this value, because they all have the ``errordef`` attribute set.
 |      
 |      To make user code more readable, we provided two named constants::
 |      
 |          m_lsq = Minuit(a_least_squares_function)
 |          m_lsq.errordef = Minuit.LEAST_SQUARES  # == 1

```

In [ ]:
# Parameters for each peak
# Change to parameters in water
nr = { "amplitude": 0.10399,
       "phase": np.pi }
r0 = { "amplitude" : 0.5,
       "pos" : 2800.,
       "width" : 20 }
r1 = { "amplitude": 0.5,
       "pos": 2950,
       "width": 20 }
r2 = { "amplitude": 1,
       "pos": 3200,
       "width": 40 }

Generate the full dictionary of parameters

In [ ]:
fitting_dictionaries = { 'nr' : nr, 'r0' : r0, 'r1' : r1, 'r2' : r2 }
parameters = {}
for label, dictionary in fitting_dictionaries.items() :
  new = { label+'_'+k:v for k, v in dictionary.items()}
  parameters = {**parameters,**new}
print(parameters)

In [ ]:
fit = Minuit(costfunction, name=parameters.keys(), *parameters.values())

In [ ]:
# Ranges should only be positive
if 'nr_amplitude' in parameters:
  fit.limits["nr_amplitude"] = (0, None)
  fit.limits["nr_phase"] = (0, 2*np.pi)
for i in range(maxnres):
  if 'r'+str(i)+'_amplitude' in parameters:
    fit.limits['r'+str(i)+'_amplitude'] = (0, None)
    fit.limits['r'+str(i)+'_pos'] = (WMin, WMax)
    fit.limits['r'+str(i)+'_width'] = (0, None)

In [ ]:
# In case we want to keep a parameter fixed
fit.fixed["nr_phase"] = False

In [ ]:
# perform the fit
fit.migrad()

In [ ]:
optimized_parameters = dict(zip(parameters.keys(),[p.value for p in fit.params]))
optimized_parameters

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcamplitude(*optimized_parameters.values()))
plt.plot(filtered_data['Wavenumbers'],filtered_data['SFG'])

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcimaginary(*optimized_parameters.values()))

## Currying the cost function
This approach allows to associate the data (wavenumber and sfg) to the cost function so that there is no potential issue of which data is used in the fit

In [ ]:
from typing import Callable
def curry(data: np.ndarray, func: Callable) -> Callable :

  def curriedfunc(*args):
    return func(data, *args)

  return curriedfunc

def costfunction_of_sfg(sfg: np.ndarray[np.float64], *args) -> np.float64 :
  return np.sum((sfg - calcamplitude(*args))**2)

def calcamplitude_of_wavenumbers(wavenumbers: np.ndarray[np.float64], *args) -> np.ndarray[np.float64] :
  Chi = np.zeros(wavenumbers.shape,dtype=np.complex128)
  Chi = Chi + chi_non_resonant(args[0], args[1])
  nres = (len(args)-2)//3
  for i in range(nres):
    iarg = 3*i+2
    ChiR = chi_resonant(wavenumbers, args[iarg], args[iarg+1], args[iarg+2])
    Chi = Chi + ChiR
  return np.square(Chi.real) + np.square(Chi.imag)

def calcimaginary_of_wavenumbers(wavenumbers: np.ndarray[np.float64], *args) -> np.ndarray[np.float64]:
  Chi = np.zeros(wavenumbers.shape,dtype=np.complex128)
  Chi = Chi + chi_non_resonant(args[0], args[1])
  nres = (len(args)-2)//3
  for i in range(nres):
    iarg = 3*i+2
    ChiR = chi_resonant(wavenumbers, args[iarg], args[iarg+1], args[iarg+2])
    Chi = Chi + ChiR
  return Chi.imag

Whenever we filter the data and generate the two arrays of `wavenumbers` and `sfg`, we should execute the following cell to create the corresponding functions in that interval

In [ ]:
calcamplitude = curry(wavenumbers,calcamplitude_of_wavenumbers)
calcimaginary = curry(wavenumbers,calcimaginary_of_wavenumbers)
costfunction = curry(sfg,costfunction_of_sfg)
costfunction.errordef = Minuit.LEAST_SQUARES

In [ ]:
fit = Minuit(costfunction, name=parameters.keys(), *parameters.values())

In [ ]:
# Ranges should only be positive
if 'nr_amplitude' in parameters:
  fit.limits["nr_amplitude"] = (0, None)
  fit.limits["nr_phase"] = (0, 2*np.pi)
for i in range((len(parameters)-2)//3):
  if 'r'+str(i)+'_amplitude' in parameters:
    fit.limits['r'+str(i)+'_amplitude'] = (0, None)
    fit.limits['r'+str(i)+'_pos'] = (WMin, WMax)
    fit.limits['r'+str(i)+'_width'] = (0, None)

In [ ]:
# perform the fit
fit.migrad()
optimized_parameters = dict(zip(parameters.keys(),[p.value for p in fit.params]))

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcamplitude(*optimized_parameters.values()))
plt.plot(filtered_data['Wavenumbers'],filtered_data['SFG'])

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcimaginary(*optimized_parameters.values()))

## Combine everything to fit six resonances

In [ ]:
# @title Resize the window of the spectrum { display-mode: "form" }
WMin = 2700 # @param {type:"number"}
WMax  = 3800 # @param {type:"number"}
filtered_data = data.query(f'Wavenumbers > {WMin} and Wavenumbers < {WMax}').copy()
filtered_data.plot('Wavenumbers','SFG')
wavenumbers = filtered_data['Wavenumbers'].values
sfg = filtered_data['SFG'].values

In [ ]:
calcamplitude = curry(wavenumbers,calcamplitude_of_wavenumbers)
calcimaginary = curry(wavenumbers,calcimaginary_of_wavenumbers)
costfunction = curry(sfg,costfunction_of_sfg)
costfunction.errordef = Minuit.LEAST_SQUARES

In [ ]:
# Parameters for each peak
# Change to parameters in water
nr = { "amplitude": 0.10399,
       "phase": np.pi }
r0 = { "amplitude" : 3,
       "pos" : 2870.,
       "width" : 20 }
r1 = { "amplitude": 7,
       "pos": 2800,
       "width": 50 }
r2 = { "amplitude": 4,
       "pos": 2780,
       "width": 20 }
r3 = { "amplitude": 1,
       "pos": 3100,
       "width": 50 }
r4 = { "amplitude": 1,
       "pos": 3350,
       "width": 50 }
r5 = { "amplitude": 1,
       "pos": 3500,
       "width": 50 }

resonant_list = [r0, r1, r2, r3, r4, r5]

In [ ]:
# Let's assume that we will always have a nonresonant dictionary plus a list of resonant dictionaries
# NOTE: this function allows to have different names for the resonant peak dictionaries
# and we can select which ones we add when we call the function
def combine_params( nonresonant_params: dict, resonant_list: list[dict] ) -> dict :
  # start with the nonresonant parameters
  parameters = { 'nr_'+k: v for k,v in nonresonant_params.items() }
  # add the resonant parameters naming them r1_ , r2_ , r3_, ...
  nres = len(resonant_list)
  for i, resonant_params in enumerate(resonant_list):
    new = {'r'+str(i)+'_'+k:v for k,v in resonant_params.items() }
    parameters = {**parameters, **new}
  return parameters

In [ ]:
combine_params( nr, [r0, r1, r2, r3, r4, r5]) # NOTE: the names in this list may not correspond to the final names of the parameters, if you change the order

In [ ]:
parameters = combine_params(nr, resonant_list)
print(parameters)

In [ ]:
fit = Minuit(costfunction, name=parameters.keys(), *parameters.values())

In [ ]:
# Ranges should only be positive
if 'nr_amplitude' in parameters:
  fit.limits["nr_amplitude"] = (0, None)
  fit.limits["nr_phase"] = (0, 2*np.pi)
for i in range((len(parameters)-2//3)):
  if 'r'+str(i)+'_amplitude' in parameters:
    fit.limits['r'+str(i)+'_amplitude'] = (0, None)
    fit.limits['r'+str(i)+'_pos'] = (WMin, WMax)
    fit.limits['r'+str(i)+'_width'] = (0, None)

In [ ]:
fit.migrad()

In [ ]:
optimized_parameters = dict(zip(parameters.keys(),[p.value for p in fit.params]))

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcamplitude(*optimized_parameters.values()))
plt.plot(filtered_data['Wavenumbers'],filtered_data['SFG'])

In [ ]:
plt.plot(filtered_data['Wavenumbers'],calcimaginary(*optimized_parameters.values()))